<a href="https://colab.research.google.com/github/abhay43/ML_Code/blob/master/DL-GAN%20Network%20in%20Face%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, Conv2DTranspose, BatchNormalization, Flatten,Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import tensorflow.keras as kr
import tensorflow as tf
import imageio
import numpy as np
import pandas as pd
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint

import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from zipfile import ZipFile
file_name = '/content/drive/My Drive/Faltu/28577_36420_compressed_fer2013.csv.zip'
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')

File Name                                             Modified             Size
fer2013.csv                                    2019-10-01 04:15:22    301072766
Extracting all the files now...
Done!


In [44]:
def load_real_samples():
  data = pd.read_csv('/content/fer2013.csv')
  pixels = list()
  pixel = list()
  for index,row in data.iterrows():
    pixel.append(list(map(int, (row['pixels'].split(' ')))) )
  for i in pixel:
    temp = list()
    for j in i:
      temp.append((j - 127.5) / 125.7)
    pixels.append(temp)
  pixel=()
  return np.array(pixels).reshape(len(pixels), 2304)

In [45]:
data = load_real_samples()

In [46]:
data.shape

(35887, 2304)

In [47]:
def build_generator(latent_dim):
  model = Sequential()
  # foundation for 4x4 image
  n_nodes = 256 * 6 * 6
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((6, 6, 256)))
  # upsample to 12x12
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # upsample to 16x16
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # upsample to 32x32
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # output layer
  model.add(Conv2D(1, (3,3), activation='tanh', padding='same'))
  model.summary()
  return model


In [48]:
def build_discriminator(in_shape=(48,48,1)):
  model = Sequential()
  # normal
  model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  # downsample
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # downsample
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # downsample
  model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.summary()
  return model


In [49]:
def generate_real_samples(dataset, n_samples):
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  # retrieve selected images
  X = dataset[ix]
  X = X.reshape((n_samples, 48, 48, 1))
  # generate 'real' class labels (1)
  y = ones((n_samples, 1))
  return X, y


In [50]:
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [51]:
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [52]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [53]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		plt.subplot(n, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i].reshape(48, 48))
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

In [54]:
def plotGeneratedImages(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
 # prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)
  

In [55]:
def train(g_model, d_model, gan_model, dataset, latent_dim, epochs=1, batchSize=128):
	batchCount = int(dataset.shape[0] / batchSize)
	half_batch = int(batchSize / 2)
	print ('Epochs:', epochs)
	print ('Batch size:', batchSize)
	print ('Batches per epoch:', batchCount)
	ss =1
	for e in range(1, epochs+1):
		print ('-'*15, 'Epoch %d' % e, '-'*15)        
		for _ in range(batchCount):                        
			ss = ss+1
			# Get a random set of input noise and images
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			print(X_real.shape)
			print(y_real.shape)
			#update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, batchSize)
			# create inverted labels for the fake samples
			y_gan = ones((batchSize, 1))
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			print('batch running-'+str(ss)+' '+ str(d_loss1) + ' ' +str(d_loss2) + ' '+str(g_loss))
		if e == 1 or e % 5 == 0:
			plotGeneratedImages(e, g_model, d_model, dataset, latent_dim)

In [56]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = build_discriminator()
# create the generator
g_model = build_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 256)        

In [ ]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, 20)